In [3]:
import pandas as pd
import requests
import time
import plotly.express as px


In [2]:
city_df=pd.read_csv('city_information.csv')
hotel_df=pd.read_csv('hotel_information.csv')

In [46]:
best_hotels_per_city = hotel_df.groupby('city', group_keys=False).apply(
    lambda x: x.sort_values('rating', ascending=False).head(20)
)

/tmp/ipykernel_535808/2363642300.py:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



### Collecting weather datas

In [ ]:
api_key = '4c707e675287a739bd57c2e2297cd975'


base_url = 'http://api.openweathermap.org/data/2.5/forecast'


def fetch_weather_data(lat, lon):
    params = {
        'lat': lat,
        'lon': lon,
        'units': 'metric',
        'appid': api_key
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data for coordinates ({lat}, {lon}): {response.status_code} - {response.text}")
        return None


weather_data = []


for index, row in city_df.iterrows():
    print(f"Fetching weather data for {row['city_name']}...")
    data = fetch_weather_data(row['city_latitude'], row['city_longitude'])
    if data and "list" in data:
        for forecast in data['list']:
            weather_data.append({
                'city': row['city_name'],
                'latitude': row['city_latitude'],
                'longitude': row['city_longitude'],
                'datetime': pd.to_datetime(forecast['dt'], unit='s'),
                'temperature': forecast['main']['temp'],
                'rain': forecast.get('rain', {}).get('3h', 0)  
            })
    time.sleep(1)  

weather_df = pd.DataFrame(weather_data)


weather_summary = weather_df.groupby('city').agg({
    'latitude': 'first',
    'longitude': 'first',
    'temperature': 'mean',
    'rain': 'sum'
}).reset_index()



Fetching weather data for Mont Saint Michel...
Fetching weather data for St Malo...
Fetching weather data for Bayeux...
Fetching weather data for Le Havre...
Fetching weather data for Rouen...
Fetching weather data for Paris...
Fetching weather data for Amiens...
Fetching weather data for Lille...
Fetching weather data for Strasbourg...
Fetching weather data for Chateau du Haut Koenigsbourg...
Fetching weather data for Colmar...
Fetching weather data for Eguisheim...
Fetching weather data for Besancon...
Fetching weather data for Dijon...
Fetching weather data for Annecy...
Fetching weather data for Grenoble...
Fetching weather data for Lyon...
Fetching weather data for Gorges du Verdon...
Fetching weather data for Bormes les Mimosas...
Fetching weather data for Cassis...
Fetching weather data for Marseille...
Fetching weather data for Aix en Provence...
Fetching weather data for Avignon...
Fetching weather data for Uzes...
Fetching weather data for Nimes...
Fetching weather data for A

### Creating a xeather score that increase with temperature and decrease with rain volume

In [ ]:

temp_avg = weather_summary['temperature'].mean()
rain_avg = weather_summary['rain'].mean()


weather_summary['normalized_temp'] = (weather_summary['temperature'] - temp_avg) / temp_avg
weather_summary['normalized_rain'] = (weather_summary['rain'] - rain_avg) / rain_avg
weather_summary['weather_score'] = weather_summary['normalized_temp'] - weather_summary['normalized_rain']+2
weather_summary['weather_score'] = weather_summary['weather_score'].clip(lower=0)



In [28]:
fig = px.scatter_mapbox(
    weather_summary,
    lat='latitude',
    lon='longitude',
    size='weather_score',
    color='weather_score',
    hover_name='city',
    hover_data={'latitude': False, 'longitude': False, 'temperature': True, 'rain': True},
    title='Weather Score for the Next 7 Days',
    labels={'weather_score': 'Weather Score'},
    mapbox_style="carto-positron",
    size_max=15,
    zoom=5
)

fig.update_layout(title="Weather Score for the Next 7 Days", coloraxis_colorbar=dict(title="Weather Score"), width=1000,height=900)
fig.show()


In [47]:
best_destinations = weather_summary.sort_values(by='weather_score', ascending=False).head(5)

In [48]:
hotels_in_best_destinations = best_hotels_per_city[best_hotels_per_city['city'].isin(best_destinations['city'])]

In [ ]:
top_destinations_cities = best_destinations['city'].tolist()
hotels_in_best_destinations = hotel_df[hotel_df['city'].isin(top_destinations_cities)]
hotels_in_best_destinations = hotels_in_best_destinations.dropna(subset=['rating', 'latitude', 'longitude'])

hotels_in_best_destinations['url_text'] = 'Link: ' + hotels_in_best_destinations['page_url']


fig = px.scatter_mapbox(
    hotels_in_best_destinations,
    lat='latitude',  
    lon='longitude',  
    color='rating',
    size='rating',
    hover_name='hotel_name',
    hover_data={'city': True, 'rating': True, 'url_text': True},  
    title='Best Hotels in the Top 5 Destinations',
    labels={'rating': 'Hotel Rating'},
    mapbox_style="carto-positron",
    size_max=15,
    zoom=5
)


fig.update_layout(
    coloraxis_colorbar=dict(title="Rating")
)

fig.show()